# Loading Libraries

In [1]:
import pandas as pd
import numpy as np

import surprise
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k


# Reading Data

In [2]:
# Read Train & Test Data
train = pd.read_csv("../../00_Data/online_retail_ratings_train.csv", index_col=0)
test = pd.read_csv("../../00_Data/online_retail_ratings_test.csv", index_col=0)

In [3]:
# Check Shapes
print("Shape of Train:", train.shape)
print("Shape of Test: \t", test.shape)

Shape of Train: (357692, 3)
Shape of Test: 	 (89479, 3)


In [4]:
# Check Head of Train
train.head()

,StockCode,CustomerID,purchased
125778,1526,2425,1
370557,1551,223,-1
440452,2043,2392,-1
428223,1756,1936,-1
285547,2043,69,-1


# Preparation of Train Dataset for Surprise Models


In [5]:
# Reader 
reader = surprise.Reader(rating_scale=(-2,2))

# Build Train Set from Custom Dataset
train_set = surprise.Dataset.load_from_df(train[['CustomerID', 'StockCode', 'purchased']], reader=reader).build_full_trainset()

# Baseline Model

In [6]:
# base_model
try:
    del(base_model)
except:
    pass

# Create Object for base_model 
base_model = surprise.SVD(random_state=0, n_factors=200, n_epochs=20, verbose=True)

# Fit the base_model
base_model.fit(train_set)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


# Prediction: Test Set Only

In [7]:
# Make Predictions
test_pred = predict(base_model, test, usercol='CustomerID', itemcol='StockCode')

# Show first five rows
test_pred.head()

,CustomerID,StockCode,prediction
0,1347,169,-1.132771
1,3672,1017,1.315107
2,2815,1553,-0.787755
3,1481,1092,-1.007120
4,26,1513,1.253311


In [8]:
# Check Statistics of Prediction
test_pred.prediction.describe()

count    89479.000000
mean         0.216805
std          0.976660
min         -2.000000
25%         -0.800308
50%          0.602584
75%          1.115199
max          2.000000
Name: prediction, dtype: float64

In [9]:
# Convert Predictions
test_pred['prediction'] = np.where((test_pred['prediction']>0),1,0)

# Convert Test
test['purchased'] = np.where((test['purchased']>0),1,0)

In [10]:
# Check Distribution
test_pred['prediction'].value_counts()

1    48189
0    41290
Name: prediction, dtype: int64

In [11]:
# Sort Index of both Datasets to use Accuracy Score 
test = test.sort_values(by=['CustomerID', 'StockCode'])
test_pred = test_pred.sort_values(by=['CustomerID', 'StockCode'])

# Reset indeces for both DataFrames
test = test.reset_index(drop=True)
test_pred = test_pred.reset_index(drop=True)

In [12]:
# First Reorder columns for test 
test = test[['CustomerID', 'StockCode', 'purchased']]
# Head of Test
test.head()

,CustomerID,StockCode,purchased
0,1,262,0
1,1,275,1
2,1,320,1
3,1,425,0
4,1,892,0


In [13]:
# Head of Test_pred
test_pred.head()

,CustomerID,StockCode,prediction
0,1,262,0
1,1,275,1
2,1,320,1
3,1,425,0
4,1,892,0


In [14]:
# Import Accuracy 
from sklearn.metrics import accuracy_score

# Accuracy 
accuracy_score(test.purchased, test_pred.prediction)

0.9350015087338929

# Prediction: Top N 

In [15]:
# Predict all pairs that are not in the train set
predictions = compute_ranking_predictions(base_model, train, usercol='CustomerID', itemcol='StockCode', remove_seen=True)

In [16]:
# Check first five rows
predictions.head()

,CustomerID,StockCode,prediction
357692,2425,1931,0.189052
357693,2425,197,0.680960
357694,2425,1592,0.781600
357695,2425,43,0.767388
357696,2425,166,0.827796


In [ ]:
# Filter Test for purchased items only
test = test[test['purchased']==1]

In [17]:
# Now filter out top N 
# First write a function
def filter_top_n(predictions: pd.DataFrame, n: int) -> pd.DataFrame:
    
    # Group the dataframe by 'CustomerID', and for each group, sort by 'prediction' in descending order, then take the top N rows
    top_n_per_customer = predictions.groupby('CustomerID', group_keys=False).apply(lambda group: group.sort_values('prediction', ascending=False).head(n))
    
    return top_n_per_customer

# Filter Top 10 
top_10 = filter_top_n(predictions, 10)

# Filter Top 20 
top_20 = filter_top_n(predictions, 20)


In [18]:
# Evaluate Precision at 10 
eval_precision_10 = precision_at_k(test, top_10, col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=10)
print('precision at 10 \t:', eval_precision_10)

# Evaluate Recall at 10 
eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=10)
print('recall at 10 \t:', eval_recall_10)

precision at 10 	: 0.01658549783549784
recall at 10 	: 0.005790876376200717


In [19]:
# Evaluate Precision at 20 
eval_precision_20 = precision_at_k(test, top_20, col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=20)
print('precision at 20 \t:', eval_precision_20)

# Evaluate Recall at 20 
eval_recall_20 = recall_at_k(test, top_20,col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=20)
print('recall at 20 \t:', eval_recall_20)

precision at 20 	: 0.014434523809523812
recall at 20 	: 0.010032832426553543


# Hyper-Parameter Tuning

In [20]:
# Define Set of Hyperparameters: 
n_factors = [5, 10, 100, 200, 1000]

# Initiate List for Metrics
factors = []
recall_k = []

# Start for loop
for factor in n_factors:
    # First Delete Model Everytime
    try:
        del(model)
    except:
        pass

    # Define Model with Parameters 
    model = surprise.SVD(random_state=0, n_factors=factor, n_epochs=10)

    # Fit model
    model.fit(train_set)

    # Predict all pairs that are not in the train set
    predictions = compute_ranking_predictions(model, test, usercol='CustomerID', itemcol='StockCode')

    # Filter Top 10 
    top_10 = filter_top_n(predictions, 10)

    # Evaluate Recall at 10 
    eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                        col_rating="purchased", col_prediction="prediction", 
                                        relevancy_method="top_k", k=10)

    print("recall at for", factor, "factors: \t", eval_recall_10)

    # Append Lists
    factors.append(factor)
    recall_k.append(eval_recall_10)

recall at for 5 factors: 	 0.004932809872165423
recall at for 10 factors: 	 0.004949100469906244
recall at for 100 factors: 	 0.004032864174746481
recall at for 200 factors: 	 0.003984895853339564
recall at for 1000 factors: 	 0.0023859926771086317


# Tuning 2

In [21]:
# Define Set of Hyperparameters: 
n_factors = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# Initiate List for Metrics
factors = []
recall_k = []

# Start for loop
for factor in n_factors:
    # First Delete Model Everytime
    try:
        del(model)
    except:
        pass

    # Define Model with Parameters 
    model = surprise.SVD(random_state=0, n_factors=factor, n_epochs=10)

    # Fit model
    model.fit(train_set)

    # Predict all pairs that are not in the train set
    predictions = compute_ranking_predictions(model, test, usercol='CustomerID', itemcol='StockCode')

    # Filter Top 10 
    top_10 = filter_top_n(predictions, 10)

    # Evaluate Recall at 10 
    eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                        col_rating="purchased", col_prediction="prediction", 
                                        relevancy_method="top_k", k=10)

    print("recall at for", factor, "factors: \t", eval_recall_10)

    # Append Lists
    factors.append(factor)
    recall_k.append(eval_recall_10)

recall at for 2 factors: 	 0.005051569550235341
recall at for 3 factors: 	 0.005009520653804825
recall at for 4 factors: 	 0.00519414964360274
recall at for 5 factors: 	 0.004932809872165423
recall at for 6 factors: 	 0.004660203462397755
recall at for 7 factors: 	 0.004796645584363985
recall at for 8 factors: 	 0.00480703302181502
recall at for 9 factors: 	 0.004831685831554745
recall at for 10 factors: 	 0.004949100469906244
recall at for 11 factors: 	 0.0046483381054583505
recall at for 12 factors: 	 0.004684334456336125
recall at for 13 factors: 	 0.004651341363759494
recall at for 14 factors: 	 0.00483968291763449
recall at for 15 factors: 	 0.004749231114270439


# Best Model: Accuracy on Test

In [28]:
# Create Object for Model
best_model = surprise.SVD(random_state=0, n_factors=4, n_epochs=20)

# Fit the base_model
best_model.fit(train_set)

# Make Predictions
test_pred = predict(best_model, test, usercol='CustomerID', itemcol='StockCode')

# Convert Predictions
test_pred['prediction'] = np.where((test_pred['prediction']>0), 1,0)

# Sort Index of both Datasets to use Accuracy Score 
test = test.sort_values(by=['CustomerID', 'StockCode'])
test_pred = test_pred.sort_values(by=['CustomerID', 'StockCode'])

# Reset indeces for both DataFrames
test = test.reset_index(drop=True)
test_pred = test_pred.reset_index(drop=True)

# Accuracy 
accuracy_score(test.purchased, test_pred.prediction)

0.9320734474010661

# Best Model: Ranking Metrics

In [29]:
# Predict All Pairs of Users & Items that are NOT in the Trainset 
predictions = compute_ranking_predictions(best_model, test, usercol='CustomerID', itemcol='StockCode')

# Filter Top 10 & 20 
top_10 = filter_top_n(predictions, 10)
top_20 = filter_top_n(predictions, 20)

# Evaluate Recall at 10 
eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                            col_rating="purchased", col_prediction="prediction", 
                            relevancy_method="top_k", k=10)
# Evaluate Recall at 20 
eval_recall_20 = recall_at_k(test, top_20,col_user="CustomerID", col_item="StockCode",
                    col_rating="purchased", col_prediction="prediction", 
                    relevancy_method="top_k", k=20)

# Evaluate Precision at 10 
eval_precision_10 = precision_at_k(test, top_10, col_user="CustomerID", col_item="StockCode",
                    col_rating="purchased", col_prediction="prediction", 
                    relevancy_method="top_k", k=10)
# Evaluate Precision at 20 
eval_precision_20 = precision_at_k(test, top_20, col_user="CustomerID", col_item="StockCode",
                    col_rating="purchased", col_prediction="prediction", 
                    relevancy_method="top_k", k=20)

print('Recall@10:\t', eval_recall_10)
print('Recall@20:\t', eval_recall_20) 
print('Precision@10:\t', eval_precision_10)
print('Precision@20:\t', eval_precision_20)

Recall@10:	 0.0054336758082322925
Recall@20:	 0.009875799624900133
Precision@10:	 0.015016233766233766
Precision@20:	 0.0137987012987013
